## Execute XGBoost Machine Learning Analysis


In [ ]:
# Libraries
import pandas as pd
import numpy as np
import os
import warnings
from pathlib import Path
from datetime import datetime
import logging

# Core ML libraries
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix
)
from sklearn.preprocessing import LabelEncoder

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# ───────────── LOAD SELECTED ROIs (MOTOR/BASAL/EXECUTIVE) ─────────────
ROI_FILE = Path("/atlas_gt/roi_labels_enhanced.csv")
roi_df = pd.read_csv(ROI_FILE)

# Reconstruct TARGET_ROIS exactly as in bootstrap script
motor_rois = roi_df[
    (roi_df['functional_system'] == 'Motor') |
    (roi_df['sub_system'].isin(['Primary', 'Premotor', 'Supplementary', 'Eye']))
]['roi_name'].tolist()

basal_rois = roi_df[
    (roi_df['functional_system'] == 'BasalGanglia') |
    (roi_df['region_full_name'].str.contains(r'Putamen|Caudate|Globus Pallidus|Nucleus Accumbens', case=False, na=False))
]['roi_name'].tolist()

executive_rois = roi_df[
    (roi_df['functional_system'] == 'Frontoparietal') &
    (roi_df['sub_system'].isin(['DLPFC', 'IFJ', 'IFS', 'VLPFC']))
]['roi_name'].tolist()

TARGET_ROIS = sorted(set(motor_rois + basal_rois + executive_rois))
N_ROIS = len(TARGET_ROIS)
ROI_NAMES = TARGET_ROIS

print(f"✅ Using {N_ROIS} targeted ROIs for analysis.")

class ConnectivityAnalyzer:
    """Clean XGBoost connectivity analyzer with direct .npy loading."""
    
    def __init__(self, output_dir="results", random_state=42):
        self.output_dir = Path(output_dir)
        self.figures_dir = self.output_dir / "figures"
        self.tables_dir = self.output_dir / "tables"
        self.logs_dir = self.output_dir / "logs"
        self.random_state = random_state
        
        for directory in [self.output_dir, self.figures_dir, self.tables_dir, self.logs_dir]:
            directory.mkdir(parents=True, exist_ok=True)
        
        self._setup_logging()
        self.band_results = {}
        self.stability_results = {}
        self.log("Connectivity Analyzer initialized")
    
    def _setup_logging(self):
        log_file = self.logs_dir / f"analysis_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
        formatter = logging.Formatter('%(asctime)s | %(levelname)-8s | %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
        
        self.logger = logging.getLogger('ConnectivityAnalyzer')
        self.logger.setLevel(logging.INFO)
        
        file_handler = logging.FileHandler(log_file)
        file_handler.setFormatter(formatter)
        self.logger.addHandler(file_handler)
        
        console_handler = logging.StreamHandler()
        console_handler.setFormatter(formatter)
        self.logger.addHandler(console_handler)
        
        self.logger.info(f"Logging initialized - Log file: {log_file}")
    
    def log(self, message, level='info'):
        getattr(self.logger, level)(message)
    
    def vectorize_upper_triangle(self, matrix):
        """Convert symmetric matrix to 1D upper triangle vector."""
        if matrix.shape != (N_ROIS, N_ROIS):
            raise ValueError(f"Expected ({N_ROIS}, {N_ROIS}) matrix, got {matrix.shape}")
        iu = np.triu_indices(N_ROIS, k=1)
        return matrix[iu]
    
    def load_group_connectivity_data_from_npy(self, features_base_dir, band_name):
        """
        Load all subjects' full + bootstrap matrices and create ML dataset.
        Returns X, y, feature_names, metadata
        """
        subject_dirs = [d for d in Path(features_base_dir).iterdir() if d.is_dir() and d.name.startswith('Sbj')]
        if not subject_dirs:
            self.log(f"No subject directories found in {features_base_dir}", 'error')
            return None, None, None, None
        
        self.log(f"Loading {len(subject_dirs)} subjects for {band_name} band")
        all_features = []
        all_labels = []
        all_metadata = []
        feature_names = None
        
        for subj_dir in sorted(subject_dirs):
            for condition, label in [('inphase', 0), ('outphase', 1)]:
                # Load full matrix
                full_path = subj_dir / f"full_{condition}_{band_name}.npy"
                if full_path.exists():
                    try:
                        mat = np.load(full_path)
                        feat = self.vectorize_upper_triangle(mat)
                        if feature_names is None:
                            iu = np.triu_indices(N_ROIS, k=1)
                            feature_names = [f"{ROI_NAMES[i]} ↔ {ROI_NAMES[j]}" for i, j in zip(iu[0], iu[1])]
                        all_features.append(feat)
                        all_labels.append(label)
                        all_metadata.append({'subject': subj_dir.name, 'type': 'full'})
                    except Exception as e:
                        self.log(f"Failed to load {full_path}: {e}", 'warning')
                
                # Load bootstrap matrices
                boot_path = subj_dir / f"bootstrap_{condition}_{band_name}.npy"
                if boot_path.exists():
                    try:
                        boot_tensor = np.load(boot_path)
                        if boot_tensor.ndim != 3:
                            self.log(f"Invalid bootstrap shape in {boot_path}: {boot_tensor.shape}", 'warning')
                            continue
                        for boot_idx in range(boot_tensor.shape[0]):
                            feat = self.vectorize_upper_triangle(boot_tensor[boot_idx])
                            if feature_names is None:
                                iu = np.triu_indices(N_ROIS, k=1)
                                feature_names = [f"{ROI_NAMES[i]} ↔ {ROI_NAMES[j]}" for i, j in zip(iu[0], iu[1])]
                            all_features.append(feat)
                            all_labels.append(label)
                            all_metadata.append({
                                'subject': subj_dir.name,
                                'type': 'bootstrap',
                                'bootstrap_id': boot_idx
                            })
                    except Exception as e:
                        self.log(f"Failed to load {boot_path}: {e}", 'warning')
        
        if not all_features:
            self.log(f"No valid data loaded for {band_name}", 'warning')
            return None, None, None, None
        
        X = np.array(all_features)
        y = np.array(all_labels)
        self.log(f"Loaded {X.shape[0]} samples × {X.shape[1]} features for {band_name}")
        return X, y, feature_names, all_metadata
    
    def _check_gpu(self):
        try:
            import subprocess
            result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
            return result.returncode == 0
        except:
            return False
    
    def compute_feature_stability(self, X, y, feature_names, band_name, n_runs=100, top_k=15):
        self.log(f"Computing feature stability for {band_name} ({n_runs} runs)")
        le = LabelEncoder()
        y_encoded = le.fit_transform(y)
        
        feature_counts = {name: 0 for name in feature_names}
        all_importances = []
        
        for i in range(n_runs):
            X_train, X_test, y_train, y_test = train_test_split(
                X, y_encoded, test_size=0.2, stratify=y_encoded,
                random_state=self.random_state + i
            )
            
            clf = XGBClassifier(
                objective='binary:logistic',
                eval_metric='logloss',
                n_estimators=100,
                max_depth=5,
                learning_rate=0.1,
                random_state=self.random_state + i,
                n_jobs=1,
                tree_method='hist',
                device='cuda' if self._check_gpu() else 'cpu',
                verbosity=0
            )
            
            clf.fit(X_train, y_train)
            
            importance_df = pd.DataFrame({
                'feature': feature_names,
                'importance': clf.feature_importances_,
                'run': i
            }).sort_values('importance', ascending=False).head(top_k)
            
            all_importances.append(importance_df)
            for feature in importance_df['feature']:
                feature_counts[feature] += 1
        
        all_importances_df = pd.concat(all_importances, ignore_index=True)
        stability_df = pd.DataFrame({
            'feature': list(feature_counts.keys()),
            'stability_score': [count / n_runs for count in feature_counts.values()],
            'selection_frequency': list(feature_counts.values()),
            'band': band_name
        }).sort_values('stability_score', ascending=False)
        
        mean_importance = all_importances_df.groupby('feature')['importance'].mean()
        stability_df['mean_importance'] = stability_df['feature'].map(mean_importance).fillna(0)
        self.log(f"Stability analysis complete - Top feature: {stability_df.iloc[0]['feature']}")
        return stability_df, all_importances_df
    
    def train_xgboost_model_from_arrays(self, X, y, feature_names, band_name, plot=True):
        self.log(f"Training XGBoost model for {band_name} band", 'info')
        le = LabelEncoder()
        y_encoded = le.fit_transform(y)
        
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_encoded, test_size=0.2, stratify=y_encoded,
            random_state=self.random_state
        )
        
        gpu_available = self._check_gpu()
        device = 'cuda' if gpu_available else 'cpu'
        self.log(f"Training with {device.upper()}")
        
        clf = XGBClassifier(
            objective='binary:logistic',
            eval_metric='logloss',
            n_estimators=150,
            max_depth=6,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=self.random_state,
            n_jobs=-1,
            tree_method='hist',
            device=device,
            verbosity=0
        )
        
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        y_proba = clf.predict_proba(X_test)[:, 1]
        
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred),
            'recall': recall_score(y_test, y_pred),
            'f1': f1_score(y_test, y_pred),
            'auc': roc_auc_score(y_test, y_proba)
        }
        
        cv_scores = cross_val_score(
            clf, X, y_encoded, cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=self.random_state),
            scoring='accuracy', n_jobs=-1
        )
        
        importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': clf.feature_importances_
        }).sort_values('importance', ascending=False)
        
        stability_df, all_importances = self.compute_feature_stability(X, y, feature_names, band_name)
        
        self.log(f"{band_name} Results:")
        for metric, value in metrics.items():
            self.log(f"  {metric.title()}: {value:.3f}")
        self.log(f"  CV Score: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
        
        if plot:
            self._plot_band_results(band_name, metrics, importance_df, stability_df, y_test, y_pred, le)
        
        result = {
            'band': band_name,
            'model': clf,
            'metrics': metrics,
            'cv_scores': cv_scores,
            'importance': importance_df,
            'stability': stability_df,
            'all_importances': all_importances,
            'label_encoder': le,
            'test_data': (X_test, y_test, y_pred)
        }
        
        self.band_results[band_name] = result
        self.stability_results[band_name] = stability_df
        return result
    
    def _plot_band_results(self, band_name, metrics, importance_df, stability_df, y_test, y_pred, label_encoder):
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle(f'{band_name} Band - Analysis Results', fontsize=16, fontweight='bold')
        
        # 1. Confusion Matrix
        cm = confusion_matrix(y_test, y_pred)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=label_encoder.classes_,
                   yticklabels=label_encoder.classes_, ax=axes[0,0])
        axes[0,0].set_title('Confusion Matrix')
        axes[0,0].set_ylabel('True Label')
        axes[0,0].set_xlabel('Predicted Label')
        
        # 2. Top 10 Feature Importances
        top_features = importance_df.head(10)
        axes[0,1].barh(range(len(top_features)), top_features['importance'])
        axes[0,1].set_yticks(range(len(top_features)))
        axes[0,1].set_yticklabels([f[:30] + '...' if len(f) > 30 else f for f in top_features['feature']], fontsize=8)
        axes[0,1].set_xlabel('XGBoost Importance')
        axes[0,1].set_title('Top 10 Most Important Features')
        axes[0,1].invert_yaxis()
        
        # 3. Feature Stability
        stable_features = stability_df.head(10)
        bars = axes[1,0].bar(range(len(stable_features)), stable_features['stability_score'])
        axes[1,0].set_xlabel('Feature Rank')
        axes[1,0].set_ylabel('Stability Score')
        axes[1,0].set_title('Top 10 Most Stable Features')
        axes[1,0].set_xticks(range(len(stable_features)))
        axes[1,0].set_xticklabels([f"F{i+1}" for i in range(len(stable_features))])
        for bar, score in zip(bars, stable_features['stability_score']):
            bar.set_color(plt.cm.viridis(score))
        
        # 4. Performance Metrics
        metric_names = list(metrics.keys())
        metric_values = list(metrics.values())
        bars = axes[1,1].bar(metric_names, metric_values, color='skyblue', alpha=0.8)
        axes[1,1].set_ylabel('Score')
        axes[1,1].set_title('Classification Metrics')
        axes[1,1].set_ylim(0, 1)
        for bar, value in zip(bars, metric_values):
            axes[1,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                          f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(self.figures_dir / f'{band_name}_analysis.png', dpi=300, bbox_inches='tight')
        plt.close(fig)
        
        # Save tables
        stability_df.head(20).to_csv(self.tables_dir / f'{band_name}_stability.csv', index=False)
        importance_df.head(20).to_csv(self.tables_dir / f'{band_name}_importance.csv', index=False)
        self.log(f"Plots and tables saved for {band_name}")
    
    def compare_all_bands(self):
        if not self.band_results: return
        
        summary_data = []
        for band, res in self.band_results.items():
            summary_data.append({
                'Band': band,
                'Accuracy': res['metrics']['accuracy'],
                'Precision': res['metrics']['precision'],
                'Recall': res['metrics']['recall'],
                'F1_Score': res['metrics']['f1'],
                'AUC': res['metrics']['auc'],
                'CV_Mean': res['cv_scores'].mean(),
                'CV_Std': res['cv_scores'].std(),
                'Top_Feature_Stability': res['stability'].iloc[0]['stability_score']
            })
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_csv(self.tables_dir / 'band_comparison_summary.csv', index=False)
        self._create_comparison_figure(summary_df)
        return summary_df
    
    def _create_comparison_figure(self, summary_df):
        fig, axes = plt.subplots(2, 2, figsize=(16, 10))
        fig.suptitle('Frequency Band Comparison', fontsize=16, fontweight='bold')
        
        # Performance metrics
        metrics = ['Accuracy', 'Precision', 'Recall', 'F1_Score', 'AUC']
        x = np.arange(len(summary_df))
        width = 0.15
        for i, m in enumerate(metrics):
            axes[0,0].bar(x + i*width, summary_df[m], width, label=m, alpha=0.8)
        axes[0,0].set_xticks(x + width*2)
        axes[0,0].set_xticklabels(summary_df['Band'], rotation=45)
        axes[0,0].legend(); axes[0,0].set_ylim(0.5, 1.0)
        axes[0,0].set_title('Classification Performance')
        
        # CV stability
        axes[0,1].errorbar(range(len(summary_df)), summary_df['CV_Mean'], yerr=summary_df['CV_Std'], marker='o')
        axes[0,1].set_xticks(range(len(summary_df)))
        axes[0,1].set_xticklabels(summary_df['Band'], rotation=45)
        axes[0,1].set_title('Cross-Validation Stability')
        axes[0,1].grid(True, alpha=0.3)
        
        # Feature stability
        axes[1,0].bar(summary_df['Band'], summary_df['Top_Feature_Stability'], color='coral', alpha=0.7)
        axes[1,0].set_xticklabels(summary_df['Band'], rotation=45)
        axes[1,0].set_title('Most Stable Feature per Band')
        
        # Summary text
        axes[1,1].axis('off')
        best_band = summary_df.loc[summary_df['Accuracy'].idxmax(), 'Band']
        text = f"ANALYSIS SUMMARY\n\nBest Band: {best_band}\nAccuracy: {summary_df['Accuracy'].max():.3f}"
        axes[1,1].text(0.1, 0.9, text, transform=axes[1,1].transAxes, fontsize=12,
                      bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.8))
        
        plt.tight_layout()
        plt.savefig(self.figures_dir / 'band_comparison.png', dpi=300, bbox_inches='tight')
        plt.close(fig)
        self.log("Comparison figure created")
    
    def generate_comprehensive_report(self):
        report = f"# Professional XGBoost Connectivity Analysis Report\nGenerated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"
        if self.band_results:
            for band, res in self.band_results.items():
                report += f"### {band} Band\nAccuracy: {res['metrics']['accuracy']:.4f}\n\n"
        report_path = self.output_dir / 'COMPREHENSIVE_ANALYSIS_REPORT.md'
        with open(report_path, 'w') as f:
            f.write(report)
        self.log(f"Comprehensive report saved: {report_path}")
        return report_path


def export_for_paper(analyzer, output_dir=None):
    paper_dir = Path(output_dir or analyzer.output_dir / "paper_exports")
    paper_dir.mkdir(exist_ok=True)
    if analyzer.band_results:
        summary = pd.DataFrame([{
            'Band': b,
            'Accuracy': f"{r['metrics']['accuracy']:.3f}"
        } for b, r in analyzer.band_results.items()])
        summary.to_csv(paper_dir / 'performance_summary.csv', index=False)
    analyzer.log(f"Paper exports complete - {len(list(paper_dir.glob('*')))} files created")


def main():
    print("="*70)
    print("Professional XGBoost Connectivity Analysis Pipeline")
    print("="*70)
    
    # ⚠️ Updated: now loads from ml_features (bootstrap outputs)
    FEATURES_BASE = '/home/jaizor/jaizor/xtra/derivatives/ml_features'
    OUTPUT_DIR = '/home/jaizor/jaizor/xtra/derivatives/features/clean_professional_results'
    
    bands = ["Theta", "Alpha", "Low_Beta", "High_Beta", "Low_Gamma", "High_Gamma"]
    
    analyzer = ConnectivityAnalyzer(output_dir=OUTPUT_DIR, random_state=42)
    
    for band in bands:
        print(f"\n{'='*50}\nAnalyzing {band} band\n{'='*50}")
        X, y, feature_names, metadata = analyzer.load_group_connectivity_data_from_npy(FEATURES_BASE, band)
        if X is None: continue
        
        result = analyzer.train_xgboost_model_from_arrays(X, y, feature_names, band, plot=True)
        # ❌ Brain plotting removed (no coordinates)
    
    if analyzer.band_results:
        analyzer.compare_all_bands()
        analyzer.generate_comprehensive_report()
        export_for_paper(analyzer)
        print(f"\n🎉 ANALYSIS COMPLETE! Results in: {OUTPUT_DIR}")
    else:
        print("❌ No valid data found for any band")
    
    return analyzer


if __name__ == "__main__":
    analyzer = main()

2026-01-08 14:55:48 | INFO     | Logging initialized - Log file: /home/jaizor/jaizor/xtra/derivatives/features/clean_professional_results/logs/analysis_log_20260108_145548.log
2026-01-08 14:55:48 | INFO     | Connectivity Analyzer initialized
2026-01-08 14:55:48 | INFO     | Loading 11 subjects for Theta band
2026-01-08 14:55:48 | INFO     | Loaded 1100 samples × 2415 features for Theta
2026-01-08 14:55:48 | INFO     | Training XGBoost model for Theta band


✅ Using 70 targeted ROIs for analysis.
Professional XGBoost Connectivity Analysis Pipeline

Analyzing Theta band


2026-01-08 14:55:48 | INFO     | Training with CUDA
2026-01-08 14:56:25 | INFO     | Computing feature stability for Theta (100 runs)
2026-01-08 15:06:43 | INFO     | Stability analysis complete - Top feature: CAU-VA-lh ↔ pGP-lh
2026-01-08 15:06:43 | INFO     | Theta Results:
2026-01-08 15:06:43 | INFO     |   Accuracy: 0.927
2026-01-08 15:06:43 | INFO     |   Precision: 0.935
2026-01-08 15:06:43 | INFO     |   Recall: 0.918
2026-01-08 15:06:43 | INFO     |   F1: 0.927
2026-01-08 15:06:43 | INFO     |   Auc: 0.979
2026-01-08 15:06:43 | INFO     |   CV Score: 0.905 ± 0.019
2026-01-08 15:06:44 | INFO     | Plots and tables saved for Theta
2026-01-08 15:06:44 | INFO     | Loading 11 subjects for Alpha band
2026-01-08 15:06:44 | INFO     | Loaded 1100 samples × 2415 features for Alpha
2026-01-08 15:06:44 | INFO     | Training XGBoost model for Alpha band



Analyzing Alpha band


2026-01-08 15:06:44 | INFO     | Training with CUDA
2026-01-08 15:07:21 | INFO     | Computing feature stability for Alpha (100 runs)
2026-01-08 15:17:58 | INFO     | Stability analysis complete - Top feature: L_IFJa_ROI ↔ R_6r_ROI
2026-01-08 15:17:58 | INFO     | Alpha Results:
2026-01-08 15:17:58 | INFO     |   Accuracy: 0.886
2026-01-08 15:17:58 | INFO     |   Precision: 0.905
2026-01-08 15:17:58 | INFO     |   Recall: 0.864
2026-01-08 15:17:58 | INFO     |   F1: 0.884
2026-01-08 15:17:58 | INFO     |   Auc: 0.964
2026-01-08 15:17:58 | INFO     |   CV Score: 0.906 ± 0.024
2026-01-08 15:17:59 | INFO     | Plots and tables saved for Alpha
2026-01-08 15:17:59 | INFO     | Loading 11 subjects for Low_Beta band
2026-01-08 15:17:59 | INFO     | Loaded 1100 samples × 2415 features for Low_Beta
2026-01-08 15:17:59 | INFO     | Training XGBoost model for Low_Beta band


...


Analyzing Low_Gamma band


2026-01-08 15:40:42 | INFO     | Training with CUDA
2026-01-08 15:41:21 | INFO     | Computing feature stability for Low_Gamma (100 runs)
2026-01-08 15:52:08 | INFO     | Stability analysis complete - Top feature: L_1_ROI ↔ NAc-shell-rh
2026-01-08 15:52:08 | INFO     | Low_Gamma Results:
2026-01-08 15:52:08 | INFO     |   Accuracy: 0.905
2026-01-08 15:52:08 | INFO     |   Precision: 0.901
2026-01-08 15:52:08 | INFO     |   Recall: 0.909
2026-01-08 15:52:08 | INFO     |   F1: 0.905
2026-01-08 15:52:08 | INFO     |   Auc: 0.965
2026-01-08 15:52:08 | INFO     |   CV Score: 0.903 ± 0.028
2026-01-08 15:52:09 | INFO     | Plots and tables saved for Low_Gamma
2026-01-08 15:52:09 | INFO     | Loading 11 subjects for High_Gamma band
2026-01-08 15:52:09 | INFO     | Loaded 1100 samples × 2415 features for High_Gamma
2026-01-08 15:52:09 | INFO     | Training XGBoost model for High_Gamma band



Analyzing High_Gamma band


2026-01-08 15:52:10 | INFO     | Training with CUDA
2026-01-08 15:52:49 | INFO     | Computing feature stability for High_Gamma (100 runs)
2026-01-08 15:58:48 | INFO     | Stability analysis complete - Top feature: R_4_ROI ↔ R_6d_ROI
2026-01-08 15:58:48 | INFO     | High_Gamma Results:
2026-01-08 15:58:48 | INFO     |   Accuracy: 0.886
2026-01-08 15:58:48 | INFO     |   Precision: 0.929
2026-01-08 15:58:48 | INFO     |   Recall: 0.836
2026-01-08 15:58:48 | INFO     |   F1: 0.880
2026-01-08 15:58:48 | INFO     |   Auc: 0.949
2026-01-08 15:58:48 | INFO     |   CV Score: 0.899 ± 0.031
2026-01-08 15:58:49 | INFO     | Plots and tables saved for High_Gamma
2026-01-08 15:58:50 | INFO     | Comparison figure created
2026-01-08 15:58:50 | INFO     | Comprehensive report saved: /home/jaizor/jaizor/xtra/derivatives/features/clean_professional_results/COMPREHENSIVE_ANALYSIS_REPORT.md
2026-01-08 15:58:50 | INFO     | Paper exports complete - 1 files created



🎉 ANALYSIS COMPLETE! Results in: /home/jaizor/jaizor/xtra/derivatives/features/clean_professional_results
